In [16]:
library(tidyverse)
library(lubridate)
library(forecast) # Para modelos AR

In [82]:
df<-read_csv('df.csv',col_names = TRUE,cols(
  Ano = col_double(),
  Mes = col_double(),
  Ano_Mes = col_character(),
  Fecha = col_date(format = ""),
  Precios_Mensuales = col_double(),
  Aportes_Energia_GWh = col_double(),
  Volumen_Util_Diario_Energia_GWh = col_double(),
  Demanda_Energia_SIN_GWh = col_double(),
  ONI = col_double(),
  CEN_Total = col_double(),
  CEN_Hidro = col_double(),
  CEN_Termica = col_double()
))

# Creacion de funciones

In [63]:
oni<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    
    n_pronosticos<-df%>%filter(Fecha>=start_at)%>%select(ONI)%>%nrow
    
    
    while(any(is.na(df$ONI))){
        fila<-sum(complete.cases(df$ONI))+1
        m<-auto.arima(unlist(unname(df[1:fila-1,'ONI'])),max.q = 0,max.Q = 0,max.d = 0,max.D = 0)
        y_est<-round(as.vector(unlist(forecast(m, 24)['mean'])),1)
        df$ONI[is.na(df$ONI)]<-y_est
    }
    #df$ONI[df$Fecha>=as.Date(start_at)]<-round(runif(n_pronosticos,-0.5,0.5),1)
    
    return(df)
}
#ok

In [64]:
aportes<-function(df, start_at, rezagos_oni=12,rezagos_aportes=24){
    
    fechaini<-as.Date(start_at)
    #n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Aportes_Energia_GWh)%>%nrow
    #df$Aportes_Energia_GWh[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Aportes_Energia_GWh,na.rm = TRUE),max(df$Aportes_Energia_GWh,na.rm = TRUE)),3)

    fecha_forecast<-seq.Date(from = as.Date(fechaini),length.out = 24,by = 'month')  # Meses para iterar

    for(i in seq(length(fecha_forecast))){                                       # Inicia el pronostico un paso adelante
    
        temp<-df%>%select(Fecha, Aportes_Energia_GWh, ONI, Aportes_Normalizados) 
        rezagos_oni<-rezagos_oni                                                #Rezagos para el ONI (parametrizable)
        rezagos_aportes<-rezagos_aportes                                        #Rezagos para los Aportes (parametrizable)

        for(j in seq(rezagos_oni)){                                             #Creacion de columnas rezagadas ONI
            col_oni<-paste0('ONI_',j)                                           #   
            temp[[col_oni]]<-lag(df$ONI,j)                                      #
        }                                                                       #

        for(k in seq(rezagos_aportes)){                                         #Creacion de columnas rezagadas Aportes
            col_aportes<-paste0('Aportes_',k)                                   #
            temp[[col_aportes]]<-lag(df$Aportes_Normalizados,k)                 #
        }                                                                       #

        new_data<-temp
        
        
        
        temp<-temp%>%select(-1,-2,-3,Aportes_Normalizados,                      #Elimina Fecha, Aportes_Energia, ONI
                                    starts_with('Aportes_[0-9]'),               #Trae los rezagos de Aportes
                                    starts_with('ONI_'))                        #Trae los rezagos de ONI


        temp<-temp%>%drop_na()                                                  #Elimina las fechas con valores NA

        #Aplicamos un modelo Forward Selection
        m_FitAll<-lm(Aportes_Normalizados~.,data = temp)                        #Ajusta un modelo con todas las vars
        m<-(step(lm(Aportes_Normalizados~1,data=temp),direction = 'forward',scope = formula(m_FitAll),trace = 0))

        # Seleccionar solo variables significativas
        toselect.x<-summary(m)$coeff[,4] < 0.01                                 #Define el parametro de tolerancia
        relevant.x <- names(toselect.x)[toselect.x == TRUE]                  
        sig.formula <- as.formula(paste("Aportes_Normalizados ~",paste(relevant.x,collapse = '+'))) 

        m<-lm(formula = sig.formula,data = temp)                                #Ajusta el modelo final
        
        y_est<-round(as.vector(unlist(unname(forecast(m,h = 1,newdata = new_data%>%filter(Fecha==fecha_forecast[i]%m+%months(-1))%>%select(Fecha, one_of(names(summary(m)$coeff[-1,1])))%>%select(-Fecha))['mean']))),4)
        
        df$Aportes_Normalizados[df$Fecha==fecha_forecast[i]]<-y_est

    }
    
    return(df)
}

In [77]:
reservas<-function(df, start_at, rezagos_aportes, rezagos_oni,rezagos_reserva, rezagos_termica_hidrica){
    
    fechaini<-as.Date(start_at)
    #n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Volumen_Util_Diario_Energia_GWh)%>%nrow
    #df$Volumen_Util_Diario_Energia_GWh[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Volumen_Util_Diario_Energia_GWh,na.rm = TRUE),max(df$Volumen_Util_Diario_Energia_GWh,na.rm = TRUE)),3)
    
    fecha_forecast<-seq.Date(from = as.Date(fechaini),length.out = 24,by = 'month')  # Meses para iterar

    for(i in seq(length(fecha_forecast))){                                       # Inicia el pronostico un paso adelante
    
        temp<-df%>%select(Fecha, Aportes_Energia_GWh, ONI, Aportes_Normalizados,Reservas_Normalizadas) 
        
        rezagos_oni<-rezagos_oni                                                #Rezagos para el ONI (parametrizable)
        rezagos_aportes<-rezagos_aportes                                        #Rezagos para los Aportes (parametrizable)
        rezagos_reserva<-rezagos_reserva
        rezagos_termica_hidrica<-rezagos_termica_hidrica
        
        for(j in seq(rezagos_oni)){                                             #Creacion de columnas rezagadas ONI
            col_oni<-paste0('ONI_',j)                                           #   
            temp[[col_oni]]<-lag(df$ONI,j)                                      #
        }                                                                       #

        for(k in seq(rezagos_aportes)){                                         #Creacion de columnas rezagadas Aportes
            col_aportes<-paste0('Aportes_',k)                                   #
            temp[[col_aportes]]<-lag(df$Aportes_Normalizados,k)                 #
        }                                                                       #
        
        for(p in seq(rezagos_reserva)){                                         #Creacion de columnas rezagadas Aportes
            col_reservas<-paste0('Reservas_',p)                                 #
            temp[[col_reservas]]<-lag(df$Reservas_Normalizadas,p)               #
        }                                                                       #
        
        for(p in seq(rezagos_termica_hidrica)){                                         #Creacion de columnas rezagadas Aportes
            col_calc<-paste0('Term_Hidraulica_',p)                                 #
            temp[[col_calc]]<-lag(df$Termica_Hidraulica,p)               #
        }                                                                       #



        new_data<-temp
        
        
        
        temp<-temp%>%select(-1,-2,-3,-Aportes_Normalizados,
                                    Reservas_Normalizadas,                     #Elimina Fecha, Aportes_Energia, ONI
                                    starts_with('Aportes_[0-9]'),               #Trae los rezagos de Aportes
                                    starts_with('Reservas_[0-9]'),              #Trae los rezagos de Reservas
                                    starts_with('Term_Hidraulica_[0-9]'),       #Trae los rezagos de Term_Hidraulica (calculada)
                                    starts_with('ONI_'))                        #Trae los rezagos de ONI


        temp<-temp%>%drop_na()                                                  #Elimina las fechas con valores NA

        #Aplicamos un modelo Forward Selection
        m_FitAll<-lm(Reservas_Normalizadas~.,data = temp)                        #Ajusta un modelo con todas las vars
        m<-(step(lm(Reservas_Normalizadas~1,data=temp),direction = 'forward',scope = formula(m_FitAll),trace = 0))

        # Seleccionar solo variables significativas
        toselect.x<-summary(m)$coeff[,4] < 0.01                                 #Define el parametro de tolerancia
        relevant.x <- names(toselect.x)[toselect.x == TRUE]                  
        sig.formula <- as.formula(paste("Reservas_Normalizadas ~",paste(relevant.x,collapse = '+'))) 

        m<-lm(formula = sig.formula,data = temp)                                #Ajusta el modelo final
        y_est<-round(as.vector(unlist(unname(forecast(m,h = 1,newdata = new_data%>%filter(Fecha==fecha_forecast[i]%m+%months(-1))%>%select(Fecha, one_of(names(summary(m)$coeff[-1,1])))%>%select(-Fecha))['mean']))),4)
        
        df$Reservas_Normalizadas[df$Fecha==fecha_forecast[i]]<-y_est

    }


    
    return(df)
}

In [78]:
precios<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Precios_Mensuales)%>%nrow
    df$Precios_Mensuales[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Precios_Mensuales,na.rm = TRUE),max(df$Precios_Mensuales,na.rm = TRUE)),3)
    
    return(df)
}

In [79]:
demanda<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Demanda_Energia_SIN_GWh)%>%nrow
    df$Demanda_Energia_SIN_GWh[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Demanda_Energia_SIN_GWh,na.rm = TRUE),max(df$Demanda_Energia_SIN_GWh,na.rm = TRUE)),3)
    
    return(df)
}

In [80]:
columnas_calculadas<-function(df){
    #debo montar dem_CEN
    df$Dem_CEN<-round(df$Demanda_Energia_SIN_GWh/(as.vector(days_in_month(df$Fecha))*df$CEN_Total*24),4)
    #debo montar Term_Hidro
    df$Termica_Hidraulica<-round(df$CEN_Termica/df$CEN_Hidro,4)
    
    return(df)
}

In [69]:
normalizar_hidrologia<-function(df){
    df<-df%>%
        left_join(
                df%>%arrange(Mes)%>%group_by(Mes)%>%summarize(mean_aportes = mean(Aportes_Energia_GWh,na.rm = TRUE), 
                                                              mean_reservas = mean(Volumen_Util_Diario_Energia_GWh,na.rm = TRUE),
                                                              sd_aportes = sd(Aportes_Energia_GWh, na.rm = TRUE),
                                                              sd_reservas = sd(Volumen_Util_Diario_Energia_GWh,na.rm = TRUE)),by='Mes')%>%
    mutate(
        Aportes_Normalizados = (Aportes_Energia_GWh-mean_aportes)/sd_aportes,
        Reservas_Normalizadas = (Volumen_Util_Diario_Energia_GWh-mean_reservas)/sd_reservas)
    
    return(df)
}

# Probar funciones

In [83]:
df<-normalizar_hidrologia(df)
df<-columnas_calculadas(df)
df<-demanda(df,start_at = '2019-06-01')
df<-oni(df,start_at = '2019-06-01')
df<-aportes(df,start_at = '2019-06-01',rezagos_oni = 12,rezagos_aportes = 24)
df<-reservas(df,start_at = '2019-06-01', rezagos_oni = 12,rezagos_aportes = 24,rezagos_reserva = 24,rezagos_termica_hidrica = 12)


#df<-precios(df,start_at = '2019-06-01')

In [84]:
df%>%tail(25)#%>%select(Fecha, ONI,Aportes_Normalizados,Reservas_Normalizadas)

Ano,Mes,Ano_Mes,Fecha,Precios_Mensuales,Aportes_Energia_GWh,Volumen_Util_Diario_Energia_GWh,Demanda_Energia_SIN_GWh,ONI,CEN_Total,CEN_Hidro,CEN_Termica,mean_aportes,mean_reservas,sd_aportes,sd_reservas,Aportes_Normalizados,Reservas_Normalizadas,Dem_CEN,Termica_Hidraulica
2019,5,2019-05,2019-05-01,172.1326,6993.882,9762.744,6104.071,0.8,17.31959,11.83457,5.3176,6027.007,9605.625,1735.7382,1865.8465,0.5570397,0.08420833,0.4737,0.4493
2019,6,2019-06,2019-06-01,NA,NA,NA,4420.277,0.8,11.88597,7.97497,3.8970,5812.246,10814.714,1105.3876,1832.7577,0.1163000,-0.20440000,NA,0.4887
2019,7,2019-07,2019-07-01,NA,NA,NA,5066.676,0.7,11.88597,7.97497,3.8970,5356.389,11468.071,1090.3524,1628.1093,-0.0471000,0.08510000,NA,0.4887
2019,8,2019-08,2019-08-01,NA,NA,NA,3374.490,0.6,11.96897,8.05797,3.8970,4724.812,11611.741,645.6103,1303.1425,-0.1836000,-0.25490000,NA,0.4836
2019,9,2019-09,2019-09-01,NA,NA,NA,5636.888,0.5,12.05397,8.14297,3.8970,4098.956,11522.656,758.1609,1099.4048,-0.1103000,-0.18310000,NA,0.4786
2019,10,2019-10,2019-10-01,NA,NA,NA,3297.969,0.3,12.29927,8.23427,4.0510,4548.208,11791.016,1034.7088,1117.0774,-0.1030000,-0.44570000,NA,0.4920
2019,11,2019-11,2019-11-01,NA,NA,NA,5010.579,0.2,12.44327,8.23427,4.1950,4933.391,12320.367,1398.7955,1000.3568,0.0851000,-0.38250000,NA,0.5095
2019,12,2019-12,2019-12-01,NA,NA,NA,5902.194,0.1,12.44927,8.24027,4.1950,3587.722,11823.708,1308.8344,948.6318,0.1771000,-0.56340000,NA,0.5091
2020,1,2020-01,2020-01-01,NA,NA,NA,4655.098,0.0,12.44927,8.24027,4.1950,2387.357,10418.906,860.5643,1018.8277,0.1771000,-0.40780000,NA,0.5091
2020,2,2020-02,2020-02-01,NA,NA,NA,5911.278,-0.1,12.58267,8.26467,4.3040,1977.469,8931.573,571.9035,947.0586,0.2121000,-0.59800000,NA,0.5208


In [88]:
generar_archivos<-function(df,
                           start_at,
                           rezagos_oni,
                           rezagos_aportes,
                           rezagos_reserva,
                           rezagos_termica_hidrica,
                           n_iter,
                           folder_location){
    
    
    if(dir.exists(folder_location)){
        while(file.remove(list.files(path = folder_location,pattern = '[0-9][0-9][0-9][0-9][0-9].csv',full.names = TRUE)[1])==TRUE){
            list.files(path = folder_location,pattern = '[0-9][0-9][0-9][0-9][0-9].csv',full.names = TRUE)[1]
        }

        fechaini<-as.Date(start_at)

        for( i in seq(n_iter)){
            ini<-now()
#             #Normalizar aportes
#             #normalizar reservas
#             #Agregar variables calculadas
#             df<-precios(df,fechaini)
#             df<-oni(df,fechaini)
#             df<-aportes(df,fechaini)
#             df<-reservas(df,fechaini)
#             df<-demanda(df,fechaini)
#             df<-columnas_calculadas(df)
            df<-read_csv('df.csv',col_names = TRUE,cols(
                          Ano = col_double(),
                          Mes = col_double(),
                          Ano_Mes = col_character(),
                          Fecha = col_date(format = ""),
                          Precios_Mensuales = col_double(),
                          Aportes_Energia_GWh = col_double(),
                          Volumen_Util_Diario_Energia_GWh = col_double(),
                          Demanda_Energia_SIN_GWh = col_double(),
                          ONI = col_double(),
                          CEN_Total = col_double(),
                          CEN_Hidro = col_double(),
                          CEN_Termica = col_double()
                        ))
            
            df<-normalizar_hidrologia(df)
            df<-columnas_calculadas(df)
            df<-demanda(df,start_at = start_at)
            df<-oni(df,start_at = start_at)
            df<-aportes(df,start_at = start_at ,rezagos_oni = rezagos_oni,rezagos_aportes = rezagos_aportes)
            df<-reservas(df,start_at = start_at,
                         rezagos_oni = rezagos_oni,
                         rezagos_aportes = rezagos_aportes,
                         rezagos_reserva = rezagos_reserva,
                         rezagos_termica_hidrica = rezagos_termica_hidrica)

            df<-precios(df,start_at = start_at)

            #pronostico de variables calculadas
            write_csv(df, paste0(folder_location,str_pad(i, pad = 0, width = 5),'.csv'))
            fin<-now()
            cat(paste0(round(fin-ini,4), ' segundos ',str_pad(i, pad = 0, width = 5),'.csv'),collapse='\n')
        }
        cat(paste0(n_iter, ' archivos generados en la ruta ',folder_location))
    } else{
        cat('Error: Directorio no existe en esta máquina')
    }

}

In [89]:
generar_archivos(df,start_at = '2019-06-01', rezagos_oni = 12, 
                                             rezagos_aportes = 24,
                                             rezagos_reserva = 24,
                                             rezagos_termica_hidrica=24,
                                             n_iter = 2,
                                             folder_location = './Resultados/')

12.9915 segundos 00001.csv 
13.0905 segundos 00002.csv 
2 archivos generados en la ruta ./Resultados/

# Tareas

In [ ]:
#hacer el arx pronosticar uno a uno y tiene como entrada el ONI y se devuelven las unidades 2h
#hacer arx para reservas con oni, aportes, relacion T/H y me devuelvo 2h
#un modelo simple de precio 2h